In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## define func

## loading bottle model (target)

In [2]:
mesh_target = o3d.io.read_triangle_mesh('../dataset/model/art_melon.STL')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([mesh_target, mesh_frame])

## converting from triangle mesh to pcd

In [3]:
pcd_target = mesh_target.sample_points_uniformly(number_of_points=10000)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_target, mesh_frame])

## loading point cloud (source)

In [4]:
pcd_source = o3d.io.read_point_cloud('../dataset/d405/d405-1.ply')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

## downsampling

In [11]:
voxel_size = 0.005

# source
pcd_source_dsampled = pcd_source.voxel_down_sample(voxel_size)
#pcd_source_dsampled.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_source_dsampled])

# target (this process is not needed, specified at converting pcd)
pcd_target_dsampled = pcd_target.voxel_down_sample(voxel_size)
#pcd_target_dsampled.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_target_dsampled])

In [6]:
pcd_target = pcd_target_dsampled
pcd_source = pcd_source_dsampled

## cleaning source point cloud

In [7]:
# crop pcd with aabb (Note. z-axis looks backward of camera)
min_bound = np.asarray([-1.0, -1.0, -1.0]) # [x_min, y_min, z_min]
max_bound = np.asarray([1.0, 1.0, 1.0]) # [x_max, y_max, z_max]
aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
pcd_source_cropped = pcd_source.crop(aabb)
o3d.visualization.draw_geometries([pcd_source_cropped, mesh_frame])

In [8]:
# remove table 
plane_model, plane_inlier_indeces = pcd_source_cropped.segment_plane(distance_threshold=0.005, ransac_n=3, num_iterations=500)
pcd_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces)
pcd_plane.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_source_cropped, pcd_plane])
pcd_source_removed_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces, invert=True)
o3d.visualization.draw_geometries([pcd_source_removed_plane])

In [9]:
# remove outlier
cl, inlier_indeces = pcd_source_removed_plane.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd_source_removed_outlier = pcd_source_removed_plane.select_by_index(inlier_indeces)
o3d.visualization.draw_geometries([pcd_source_removed_outlier])

## RANSAC pipeline including feature extraction

## ICP pipeline

## show result